<a href="https://colab.research.google.com/github/FindWhatYou/data_pipelining/blob/main/1_weather_and_flight_APIs_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# City list

In [ ]:
import pandas as pd

def load_city_data(url):
    """
    Load city data from a given URL.

    Parameters:
        url (str): The URL from which to fetch the data.

    Returns:
        pandas.DataFrame: A DataFrame containing the city data.

    Example:
        >>> url = "https://drive.google.com/file/d/1gSBvbypiQwFRbdN0qVt4SULMMKCRULEc/view?usp=sharing"
        >>> city_data = load_city_data(url)
        >>> print(city_data.head())
           City Name  Population  Area (sq. km)
        0     New York     8175133          468.9
        1  Los Angeles     3792621         1214.9
        2      Chicago     2695598          606.1
        3      Houston     2100263         1552.9
        4      Phoenix     1445632         1379.6
    """
    path = "https://drive.google.com/uc?export=download&id=" + url.split("/")[-2]
    city_data = pd.read_csv(path)
    return city_data

# Example usage:
# url = "https://drive.google.com/file/d/1gSBvbypiQwFRbdN0qVt4SULMMKCRULEc/view?usp=sharing"
# city_all = load_city_data(url)


Documentation:

The code defines a function load_city_data that is used to fetch city data from a given URL and returns the data as a Pandas DataFrame. The url parameter is expected to be a string containing the URL of the CSV file that contains the city data. The function fetches the data from the URL, reads it using pd.read_csv, and returns the resulting DataFrame.

To use this function, provide the URL of the CSV file containing the city data to the load_city_data function, and it will return a DataFrame with columns "City Name", "Population", and "Area (sq. km)". The function has an example usage provided as comments at the end of the code.

Note: The code assumes that the given URL points to a valid CSV file containing city data with appropriate column names. The example usage demonstrates how to call the load_city_data function and print the first few rows of the loaded data.

Use the `city_ascii` therefore there are only basic symbols.

In [ ]:
import pandas as pd

def city_targets(url, path, pop=200000, samp=1000):
    """
    Generate a table of target cities based on population criteria.

    This function fetches city data from a given URL, filters cities based on the population threshold,
    and randomly selects a sample of cities to create a target table.

    Parameters:
        url (str): The URL of the CSV file containing the city data.
        path (str): The URL from which to fetch the data.
        pop (int, optional): The population threshold. Cities with population greater than this value will be considered. Default is 200,000.
        samp (int, optional): The number of target cities to sample. Default is 1000.

    Returns:
        pandas.DataFrame: A DataFrame containing the target cities' information, including 'city_id', 'city_ascii', 'iso3', 'lat', and 'lng'.

    Example:
        >>> url = "https://drive.google.com/file/d/1gSBvbypiQwFRbdN0qVt4SULMMKCRULEc/view?usp=sharing"
        >>> target_cities = city_targets(url, pop=500000, samp=200)
        >>> print(target_cities.head())
       city_id   city_ascii iso3      lat       lng
    0       0        Cairo  EGY  30.0626   31.2497
    1       1  Addis Ababa  ETH   9.0333   38.7000
    2       2        Accra  GHA   5.5600   -0.2057
    3       3     Ahmedabad  IND  23.0301   72.5800
    4       4       Aleppo  SYR  36.2504   37.9997
    """
    path = "https://drive.google.com/uc?export=download&id=" + url.split("/")[-2]
    city_all = pd.read_csv(path)
    city_list = city_all[city_all["population"] > pop]
    city_targets = city_list[['city_ascii', 'iso3', 'lat', 'lng']].sample(samp).reset_index().copy()
    city_targets.drop(columns='index', inplace=True)
    cities_table = city_targets.reset_index().rename(columns={'index': 'city_id'})
    return cities_table

# Example usage:
# url = "https://drive.google.com/file/d/1gSBvbypiQwFRbdN0qVt4SULMMKCRULEc/view?usp=sharing"
# target_cities = city_targets(url, pop=500000, samp=200)
# print(target_cities.head())


Documentation:

The function city_targets generates a table of target cities based on a population threshold. It fetches city data from the given URL, filters cities with populations greater than the specified threshold (pop), and then randomly samples a specified number of target cities (samp). The resulting DataFrame contains information about the target cities, including 'city_id', 'city_ascii', 'iso3', 'lat', and 'lng'.

Parameters:

url (str): The URL of the CSV file containing the city data.
path (str): The URL from which to fetch the data.
pop (int, optional): The population threshold. Cities with population greater than this value will be considered. Default is 200,000.
samp (int, optional): The number of target cities to sample. Default is 1000.
Returns:

pandas.DataFrame: A DataFrame containing the target cities' information, including 'city_id', 'city_ascii', 'iso3', 'lat', and 'lng'.
The example usage demonstrates how to call the city_targets function with a specified URL, population threshold (pop), and sample size (samp). It then prints the first few rows of the resulting target cities DataFrame.

In [ ]:
cities_table = city_targets(url,path,200000,5)

In [ ]:
cities_table

,city_id,city_ascii,iso3,lat,lng
0,0,Delmas,HTI,18.5500,-72.3000
1,1,Phoenix,USA,33.5722,-112.0892
2,2,Rome,ITA,41.8931,12.4828
3,3,Van,TUR,38.4942,43.3800
4,4,Douala,CMR,4.0500,9.7000


# APIs 1: collect weather data

In [ ]:
import requests  # For making HTTP requests
import json  # For working with JSON data
from datetime import datetime  # For dealing with date and time
import pytz  # For working with time zones
import os  # For operating system-related functionalities

In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

The code provided defines a function named weather_in_city(). This function fetches weather data from an external API for multiple cities, stores the data in a pandas DataFrame, and returns the DataFrame with weather information.



In [ ]:
def weather_in_city(con):
    """
    Fetch weather data from an API for multiple cities and store it in a DataFrame.

    The function queries an external weather API for each city listed in the 'cities' table from the given database connection.
    It collects weather information such as forecast time, outlook, temperature, feels-like temperature, wind speed,
    rain probability, and current timestamp when the API call is made. The collected data is then stored in a DataFrame.

    Parameters:
        con: The database connection object.

    Returns:
        pandas.DataFrame: A DataFrame containing weather information for the listed cities.

    Example:
        >>> import sqlite3
        >>> con = sqlite3.connect("cities.db")
        >>> weather_data = weather_in_city(con)
        >>> print(weather_data.head())
       city_id  city_name_api        forecast_time           outlook  temperature  feels_like  wind_speed  rain_prob          time_stamp
    0        1       New York  2023-08-05 06:00:00  scattered clouds        27.02       26.57        2.18       0.14 2023-08-04 15:28:45
    1        1       New York  2023-08-05 09:00:00   overcast clouds        26.85       26.51        2.30       0.11 2023-08-04 15:28:45
    2        1       New York  2023-08-05 12:00:00   overcast clouds        26.98       26.58        2.29       0.02 2023-08-04 15:28:45
    3        1       New York  2023-08-05 15:00:00   overcast clouds        27.71       27.20        2.41       0.05 2023-08-04 15:28:45
    4        1       New York  2023-08-05 18:00:00   overcast clouds        28.48       27.99        2.40       0.03 2023-08-04 15:28:45
    """
    tz = pytz.timezone('Europe/Berlin')
    now = datetime.now().astimezone(tz)

    cities_table = pd.read_sql("SELECT * FROM cities", con)

    API_key = os.environ['ENV_WEATHER_API']

    weather_df = {
        'city_id': [],
        'city_name_api': [],
        'forecast_time': [],
        'outlook': [],
        'temperature': [],
        'feels_like': [],
        'wind_speed': [],
        'rain_prob': [],
        'time_stamp': []
    }

    for i in range(len(cities_table)):
        country_code = cities_table.loc[i, "iso3"]
        city_name = cities_table.loc[i, "city_ascii"]
        lonfit = requests.get(f'http://api.openweathermap.org/data/2.5/forecast?q={city_name},{country_code}&appid={API_key}&units=metric')
        lonfit_json = lonfit.json()

        if lonfit_json['cod'] == '200':
            for j in range(40):
                weather_df['city_id'].append(cities_table.city_id[i])
                weather_df['city_name_api'].append(lonfit_json['city']['name'])
                weather_df['forecast_time'].append(lonfit_json['list'][j]['dt_txt'])
                weather_df['outlook'].append(lonfit_json['list'][j]['weather'][0]['description'])
                weather_df['temperature'].append(lonfit_json['list'][j]['main']['temp'])
                weather_df['feels_like'].append(lonfit_json['list'][j]['main']['feels_like'])
                weather_df['wind_speed'].append(lonfit_json['list'][j]['wind']['speed'])
                weather_df['rain_prob'].append(lonfit_json['list'][j]['pop'])
                weather_df['time_stamp'].append(now.strftime("%d-%m-%Y %H:%M:%S"))
        else:
            weather_df['city_id'].append(cities_table.city_id[i])
            weather_df['city_name_api'].append(city_name)
            weather_df['forecast_time'].append(None)
            weather_df['outlook'].append(lonfit_json['message'])
            weather_df['temperature'].append(None)
            weather_df['feels_like'].append(None)
            weather_df['wind_speed'].append(None)
            weather_df['rain_prob'].append(None)
            weather_df['time_stamp'].append(now.strftime("%d-%m-%Y %H:%M:%S"))

    weather_city_df = pd.DataFrame(weather_df)
    weather_city_df['time_stamp'] = pd.to_datetime(weather_city_df.time_stamp)
    return weather_city_df


In [ ]:
weather_city_df = weather_in_city()

In [ ]:
weather_city_df

,city_id,city_name_api,forecast_time,outlook,temperature,feels_like,wind_speed,rain_prob,time_stamp
0,0,Delmas 73,2023-08-03 21:00:00,light rain,32.20,37.55,4.14,0.84,2023-03-08 20:30:22
1,0,Delmas 73,2023-08-04 00:00:00,moderate rain,28.60,32.91,2.04,0.96,2023-03-08 20:30:22
2,0,Delmas 73,2023-08-04 03:00:00,moderate rain,25.71,26.43,1.62,0.99,2023-03-08 20:30:22
3,0,Delmas 73,2023-08-04 06:00:00,overcast clouds,25.77,26.47,1.42,0.72,2023-03-08 20:30:22
4,0,Delmas 73,2023-08-04 09:00:00,overcast clouds,25.37,26.08,1.32,0.00,2023-03-08 20:30:22
...,...,...,...,...,...,...,...,...,...
195,4,Douala,2023-08-08 06:00:00,light rain,23.43,24.18,0.80,0.57,2023-03-08 20:30:22
196,4,Douala,2023-08-08 09:00:00,light rain,25.83,26.53,1.17,0.87,2023-03-08 20:30:22
197,4,Douala,2023-08-08 12:00:00,light rain,28.04,30.61,3.37,0.88,2023-03-08 20:30:22
198,4,Douala,2023-08-08 15:00:00,light rain,27.91,30.12,3.55,0.64,2023-03-08 20:30:22


Check for emmpty rows

In [ ]:
weather_city_df[weather_city_df['outlook'] == 0]

,city_id,city_name_api,forecast_time,outlook,temperature,feels_like,wind_speed,rain_prob,time_stamp


# APIs 2.1: airports_of_cities

The code you provided defines a dictionary named headers, which contains two HTTP headers, "X-RapidAPI-Key" and "X-RapidAPI-Host". These headers are typically used in HTTP requests when making API calls that require authentication or identification.

In [ ]:
headers = {
	"X-RapidAPI-Key": os.environ['ENV_AIRPORT_API3'],
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

Description of the headers:

"X-RapidAPI-Key": This header is used for API key authentication. The value of this header is set to the environment variable ENV_AIRPORT_API3, which is assumed to contain the API key for accessing the airport API. The API key is typically provided by the API provider to authenticate and track usage.
"X-RapidAPI-Host": This header is used to specify the host (or domain) of the API server. In this case, it is set to "aerodatabox.p.rapidapi.com", indicating the endpoint where the airport data API is hosted.

In [ ]:
import requests
import pandas as pd

def retrive_airport(city_id, cities_table, headers):
    """
    Retrieve airport data for a given city from an external API.

    Parameters:
        city_id (int): The ID of the city in the 'cities_table'.
        cities_table (pandas.DataFrame): A DataFrame containing city information.
        headers (dict): The HTTP headers to be used for the API request.

    Returns:
        pandas.DataFrame: A DataFrame containing airport information for the city.

    Example:
        >>> city_id = 0
        >>> airport_data = retrive_airport(city_id, cities_table, headers)
        >>> print(airport_data.head())
            code                                        name      locationLat  \
        0   JFK     John F Kennedy International Airport     40.641311
        1   LGA       La Guardia Airport (Marine Air Terminal)  40.777245
        2   EWR                      Newark Liberty International  40.692500
    """

    lat = cities_table.loc[city_id, "lat"]
    lng = cities_table.loc[city_id, "lng"]
    url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
    querystring = {"lat": lat, "lon": lng, "radiusKm": "100", "limit": "10"}
    response = requests.get(url, headers=headers, params=querystring)
    airport_df = pd.json_normalize(response.json()['items'])
    airport_df["city_id"] = cities_table.city_id[city_id]
    return airport_df

def city_airports(city_id, cities_table, headers):
    """
    Retrieve airport data for multiple cities and combine the results into a single DataFrame.

    Parameters:
        city_id (int): The number of cities for which to retrieve airport data.
        cities_table (pandas.DataFrame): A DataFrame containing city information.
        headers (dict): The HTTP headers to be used for the API request.

    Returns:
        pandas.DataFrame: A DataFrame containing airport information for all cities.

    Example:
        >>> city_data = city_airports(5, cities_table, headers)
        >>> print(city_data.head())
            code                                        name      locationLat  \
        0   JFK     John F Kennedy International Airport     40.641311
        1   LGA       La Guardia Airport (Marine Air Terminal)  40.777245
        2   EWR                      Newark Liberty International  40.692500
        3   EBG     El Bagre Airport     7.5974
        4   CES     Cessnock Airport     -32.787498
    """
    flight_data = pd.DataFrame()
    for i in range(city_id):
        airport_data = retrive_airport(i, cities_table, headers)
        flight_data = pd.concat([flight_data, airport_data], axis=0)
        flight_data_df = flight_data.rename(columns={'location.lat': 'locationLat', 'location.lon': 'locationLon'})
    return flight_data_df


Description of the functions:

retrive_airport:
This function retrieves airport data for a given city using its latitude and longitude. It makes an API request to the external airport API and returns a DataFrame containing information about airports near the specified city. The function requires the city_id to identify the city in the cities_table, the cities_table itself to fetch latitude and longitude data for the city, and the headers dictionary to include necessary HTTP headers for the API request.
city_airports:
This function retrieves airport data for multiple cities by calling the retrive_airport function for each city. It concatenates the results into a single DataFrame and returns it. The function requires the total number of cities city_id, the cities_table with city information, and the headers dictionary for API authentication.
The examples provided in the documentation demonstrate how to use these functions to retrieve and consolidate airport data for specific cities. The actual usage may vary depending on the content of the cities_table and the specific API endpoint being used.

In [ ]:
flight_data_df = city_airports(5).copy()

In [ ]:
flight_data_df

,icao,iata,name,shortName,municipalityName,countryCode,locationLat,locationLon,city_id
0,RPLL,MNL,Manila Ninoy Aquino,Ninoy Aquino,Manila,PH,14.508600,121.02000,0
1,RPLS,NSP,Cavite City Sangley Point Air Base,Sangley Point Air Base,Cavite City,PH,14.495400,120.90400,0
0,ZYCH,CNI,Changhai,Changhai,Changhai,CN,39.266666,122.66695,1
1,ZYTL,DLC,Dalian Zhoushuizi,Zhoushuizi,Dalian,CN,38.965700,121.53900,1
0,ZSYW,YIW,Yiwu,Yiwu,Yiwu,CN,29.344700,120.03200,2
0,ZYJL,JIL,Jilin City Jilin,Jilin,Jilin City,CN,44.002200,126.39600,3
0,ZGZJ,ZHA,Zhanjiang,Zhanjiang,Zhanjiang,CN,21.214400,110.35800,4


# APIs 2.2: Flight Information

In [ ]:
import requests
import pandas as pd
from datetime import datetime
import pytz
import os

def flight_arrival(flight_data_df, con):
    """
    Retrieve flight arrival data from an external API for specific airports and time frames.

    The function queries an external API for flight arrival data for each airport in the provided DataFrame 'flight_data_df'
    for a specified time frame (00:00 to 11:59). The function processes the response to extract relevant flight arrival information,
    including ICAO code, arrival time, arrival terminal, departure city, departure ICAO, departure time, airline flight number, and timestamp.
    The resulting data is stored in a DataFrame and returned.

    Parameters:
        flight_data_df (pandas.DataFrame): A DataFrame containing airport information for specific cities.
        con: The database connection object.

    Returns:
        pandas.DataFrame: A DataFrame containing flight arrival information.

    Example:
        >>> airport_data = flight_arrival(flight_data_df, con)
        >>> print(airport_data.head())
           flight_id icao  arrival_time_local arrival_terminal  \
        0          0  JFK 2023-08-04 12:05:00             5FS
        1          1  JFK 2023-08-04 12:25:00             5FS
        2          2  JFK 2023-08-04 12:46:00             5FS
        3          3  LGA 2023-08-04 12:08:00              A
        4          4  LGA 2023-08-04 12:13:00              A
    """
    tz = pytz.timezone('Europe/Berlin')
    now = datetime.now().astimezone(tz)
    date = now.strftime("%Y-%m-%d")
    time_frame = ["00:00", "11:59"]

    headers = {
        "X-RapidAPI-Key": os.environ['ENV_AIRPORT_API3'],
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    flight_arrivals = {
        'icao': [],
        'arrival_time_local': [],
        'arrival_terminal': [],
        'departure_city': [],
        'departure_icao': [],
        'departure_time_local': [],
        'airline_flight_number': [],
        'time_stamp': []
    }

    for i in range(len(flight_data_df.icao)):
        icao = flight_data_df.icao.iloc[i]
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{date}T{time_frame[0]}/{date}T{time_frame[1]}"
        querystring = {"withLeg": "true", "direction": "Arrival", "withCancelled": "false", "withCodeshared": "true", "withCargo": "false", "withPrivate": "false"}
        response = requests.get(url, headers=headers, params=querystring)

        if response.status_code == 200:
            flights_json = response.json()
            for j in range(len(flights_json['arrivals'])):
                flight_arrivals['icao'].append(flight_data_df.icao.iloc[i])
                flight_arrivals['arrival_time_local'].append(flights_json['arrivals'][j]['arrival']['scheduledTimeLocal'])
                try:
                    flight_arrivals['arrival_terminal'].append(flights_json['arrivals'][j]['departure']['terminal'])
                except KeyError:
                    flight_arrivals['arrival_terminal'].append(None)
                flight_arrivals['departure_city'].append(flights_json['arrivals'][j]['departure']['airport']['name'])
                try:
                    flight_arrivals['departure_icao'].append(flights_json['arrivals'][j]['departure']['airport']['icao'])
                except KeyError:
                    flight_arrivals['departure_icao'].append(None)
                try:
                    flight_arrivals['departure_time_local'].append(flights_json['arrivals'][j]['departure']['scheduledTimeLocal'])
                except KeyError:
                    flight_arrivals['departure_time_local'].append(None)
                flight_arrivals['airline_flight_number'].append(flights_json['arrivals'][j]['number'])
                flight_arrivals['time_stamp'].append(now.strftime("%d-%m-%Y %H:%M:%S"))
        else:
            flight_arrivals['icao'].append(flight_data_df.icao.iloc[i])
            flight_arrivals['arrival_time_local'].append(None)
            flight_arrivals['arrival_terminal'].append(None)
            flight_arrivals['departure_city'].append(None)
            flight_arrivals['departure_icao'].append(None)
            flight_arrivals['departure_time_local'].append(None)
            flight_arrivals['airline_flight_number'].append(None)
            flight_arrivals['time_stamp'].append(now.strftime("%d-%m-%Y %H:%M:%S"))

    flight_arrivals_df = pd.DataFrame(flight_arrivals)

    flight_arrivals_reset = flight_arrivals_df.reset_index().copy()
    flight_arrivals_df = flight_arrivals_reset.rename(columns={"index": "flight_id"})

    flight_arrivals_df['time_stamp'] = pd.to_datetime(flight_arrivals_df.time_stamp)
    flight_arrivals_df['arrival_time_local'] = pd.to_datetime(flight_arrivals_df.arrival_time_local)
    flight_arrivals_df.dropna(subset=['departure_time_local'], inplace=True)
    flight_arrivals_df['departure_time_local'] = pd.to_datetime(flight_arrivals_df['departure_time_local'], utc=True)

    max_id = pd.read_sql("SELECT max(flight_id) FROM flight_informations", con).iat[0, 0]
    if max_id is None:
        max_id = 0
    flight_arrivals_df['flight_id'] = flight_arrivals_df.flight_id + max_id

    return flight_arrivals_df


Description of the function:

The flight_arrival() function retrieves flight arrival data from an external API for specific airports and time frames. It uses the flight_data_df DataFrame, which contains airport information for specific cities. For each airport in the flight_data_df, the function makes an API request to fetch flight arrival data within a specific time frame (00:00 to 11:59). The function then processes the API response to extract relevant flight arrival information, including ICAO code, arrival time, arrival terminal, departure city, departure ICAO, departure time, airline flight number, and timestamp. The resulting data is stored in a DataFrame and returned.

The function has two parameters:

flight_data_df: A DataFrame containing airport information for specific cities (ICAO codes).
con: The database connection object used to fetch the max flight_id from the "flight_informations" table.
Make sure to provide the correct input DataFrame flight_data_df, which should contain the ICAO codes of airports for which you want to retrieve flight arrival data. The con parameter is assumed to be a database

In [ ]:
flight_arrival_df = flight_arrival().copy()

In [ ]:
flight_arrival_df

,flight_id,icao,arrival_time_local,arrival_terminal,departure_city,departure_icao,departure_time_local,airline_flight_number,time_stamp
1,223,RPLL,2023-08-04 00:30:00+08:00,1,Hong Kong,VHHH,2023-08-03 14:05:00+00:00,5J 115,2023-04-08 21:41:21
2,224,RPLL,2023-08-04 00:15:00+08:00,None,Bacolod City,RPVB,2023-08-03 14:50:00+00:00,5J 472,2023-04-08 21:41:21
3,225,RPLL,2023-08-04 00:50:00+08:00,2,Tokyo,RJAA,2023-08-03 12:00:00+00:00,5J 5057,2023-04-08 21:41:21
4,226,RPLL,2023-08-04 00:35:00+08:00,1,Lapu-Lapu City,RPVM,2023-08-03 15:00:00+00:00,5J 574,2023-04-08 21:41:21
5,227,RPLL,2023-08-04 00:25:00+08:00,None,Davao City,RPMD,2023-08-03 14:20:00+00:00,5J 974,2023-04-08 21:41:21
...,...,...,...,...,...,...,...,...,...
217,439,ZGZJ,2023-08-04 09:25:00+08:00,1,Shanghai,ZSPD,2023-08-03 22:25:00+00:00,FM 9357,2023-04-08 21:41:21
218,440,ZGZJ,2023-08-04 09:25:00+08:00,1,Haikou,ZJHK,2023-08-04 00:35:00+00:00,GS 6525,2023-04-08 21:41:21
219,441,ZGZJ,2023-08-04 09:50:00+08:00,None,Wuhan,ZHHH,2023-08-03 23:30:00+00:00,MU 2493,2023-04-08 21:41:21
221,443,ZGZJ,2023-08-04 10:10:00+08:00,None,Kunming,ZPPP,2023-08-04 00:20:00+00:00,MU 5875,2023-04-08 21:41:21


# Push the Data the SQL DB

In [ ]:
import sqlalchemy # install if needed

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

True

### AWS  Cred

In [ ]:
schema="gans_db"
host="wbs-project5-db.civ4ous8ttqf.eu-central-1.rds.amazonaws.com"
user="admin"
password=os.environ['MY_ENV_PWD']
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

### Cities


In [ ]:
cities_table.to_sql('cities',
              if_exists='append',
              con=con,
              index=False)

5

### Weather (Lamda)

In [ ]:
weather_city_df.to_sql('weathers_table',
              if_exists='append',
              con=con,
              index=False)

200

### Airports

In [ ]:
flight_data_df.to_sql('cities_to_airport',
              if_exists='append',
              con=con,
              index=False)

7

### Flight Information (Lamda)

In [ ]:
flight_arrival_df.to_sql('flight_informations',
              if_exists='append',
              con=con,
              index=False)

184

## Testing

In [ ]:
pd.read_sql("show tables", con)

,Tables_in_gans_db
0,airport_names
1,cities
2,cities_to_airport
3,flight_informations
4,population
5,weathers_table


In [ ]:
pd.read_sql("SELECT * FROM cities", con)

,city_id,city_ascii,iso3,lat,lng
0,0,Santo Tomas,PHL,14.0833,121.183
1,1,Wafangdian,CHN,39.6271,121.997
2,2,Dongyang,CHN,29.2785,120.228
3,3,Shulan,CHN,44.4079,126.943
4,4,Wuchuan,CHN,21.4283,110.775


In [ ]:
pd.read_sql("SELECT * FROM weathers_table", con)

,city_id,city_name_api,forecast_time,outlook,temperature,feels_like,wind_speed,rain_prob,time_stamp
0,0,Delmas 73,2023-08-03 21:00:00,light rain,32.20,37.55,4.14,0.84,2023-03-08 20:30:22
1,0,Delmas 73,2023-08-04 00:00:00,moderate rain,28.60,32.91,2.04,0.96,2023-03-08 20:30:22
2,0,Delmas 73,2023-08-04 03:00:00,moderate rain,25.71,26.43,1.62,0.99,2023-03-08 20:30:22
3,0,Delmas 73,2023-08-04 06:00:00,overcast clouds,25.77,26.47,1.42,0.72,2023-03-08 20:30:22
4,0,Delmas 73,2023-08-04 09:00:00,overcast clouds,25.37,26.08,1.32,0.00,2023-03-08 20:30:22
...,...,...,...,...,...,...,...,...,...
195,4,Douala,2023-08-08 06:00:00,light rain,23.43,24.18,0.80,0.57,2023-03-08 20:30:22
196,4,Douala,2023-08-08 09:00:00,light rain,25.83,26.53,1.17,0.87,2023-03-08 20:30:22
197,4,Douala,2023-08-08 12:00:00,light rain,28.04,30.61,3.37,0.88,2023-03-08 20:30:22
198,4,Douala,2023-08-08 15:00:00,light rain,27.91,30.12,3.55,0.64,2023-03-08 20:30:22


In [ ]:
pd.read_sql("SELECT * FROM cities_to_airport", con)

,icao,iata,name,shortName,municipalityName,countryCode,locationLat,locationLon,city_id
0,RPLL,MNL,Manila Ninoy Aquino,Ninoy Aquino,Manila,P,14.5086,121.020,0
1,RPLS,NSP,Cavite City Sangley Point Air Base,Sangley Point Air Base,Cavite City,P,14.4954,120.904,0
2,ZGZJ,ZHA,Zhanjiang,Zhanjiang,Zhanjiang,C,21.2144,110.358,4
3,ZSYW,YIW,Yiwu,Yiwu,Yiwu,C,29.3447,120.032,2
4,ZYCH,CNI,Changhai,Changhai,Changhai,C,39.2667,122.667,1
5,ZYJL,JIL,Jilin City Jilin,Jilin,Jilin City,C,44.0022,126.396,3
6,ZYTL,DLC,Dalian Zhoushuizi,Zhoushuizi,Dalian,C,38.9657,121.539,1


In [ ]:
pd.read_sql("SELECT * FROM flight_informations", con)

,flight_id,icao,arrival_time_local,arrival_terminal,departure_city,departure_icao,departure_time_local,airline_flight_number,time_stamp
0,1,RPLL,2023-08-04 00:30:00,1.0,Hong Kong,VHHH,2023-08-03 14:05:00,5J 115,2023-04-08 16:03:12
1,2,RPLL,2023-08-04 00:15:00,NaN,Bacolod City,RPVB,2023-08-03 14:50:00,5J 472,2023-04-08 16:03:12
2,3,RPLL,2023-08-04 00:50:00,2.0,Tokyo,RJAA,2023-08-03 12:00:00,5J 5057,2023-04-08 16:03:12
3,4,RPLL,2023-08-04 00:35:00,1.0,Lapu-Lapu City,RPVM,2023-08-03 15:00:00,5J 574,2023-04-08 16:03:12
4,5,RPLL,2023-08-04 00:25:00,NaN,Davao City,RPMD,2023-08-03 14:20:00,5J 974,2023-04-08 16:03:12
...,...,...,...,...,...,...,...,...,...
179,217,ZGZJ,2023-08-04 09:25:00,1.0,Shanghai,ZSPD,2023-08-03 22:25:00,FM 9357,2023-04-08 16:03:12
180,218,ZGZJ,2023-08-04 09:25:00,1.0,Haikou,ZJHK,2023-08-04 00:35:00,GS 6525,2023-04-08 16:03:12
181,219,ZGZJ,2023-08-04 09:50:00,NaN,Wuhan,ZHHH,2023-08-03 23:30:00,MU 2493,2023-04-08 16:03:12
182,221,ZGZJ,2023-08-04 10:10:00,NaN,Kunming,ZPPP,2023-08-04 00:20:00,MU 5875,2023-04-08 16:03:12


In [ ]:
max_id = pd.read_sql("SELECT max(flight_id) FROM flight_informations", con).iat[0,0]
if max_id == None:
    max_id = 0

testtest_df['flight_id'] = testtest_df.flight_id + max_id
testtest_df

,flight_id,icao,arrival_time_local,arrival_terminal,departure_city,departure_icao,departure_time_local,airline_flight_number,time_stamp
0,2445,RPLL,2023-08-04 00:30:00,1.0,Hong Kong,VHHH,2023-08-03 14:05:00,5J 115,2023-04-08 21:20:05
1,2446,RPLL,2023-08-04 00:15:00,NaN,Bacolod City,RPVB,2023-08-03 14:50:00,5J 472,2023-04-08 21:20:05
2,2447,RPLL,2023-08-04 00:50:00,2.0,Tokyo,RJAA,2023-08-03 12:00:00,5J 5057,2023-04-08 21:20:05
3,2448,RPLL,2023-08-04 00:35:00,1.0,Lapu-Lapu City,RPVM,2023-08-03 15:00:00,5J 574,2023-04-08 21:20:05
4,2449,RPLL,2023-08-04 00:25:00,NaN,Davao City,RPMD,2023-08-03 14:20:00,5J 974,2023-04-08 21:20:05
...,...,...,...,...,...,...,...,...,...
179,2662,ZYTL,2023-08-04 11:40:00,NaN,Weifang,ZSWF,2023-08-04 02:30:00,HU 7739,2023-04-08 21:20:05
180,2663,ZYTL,2023-08-04 11:45:00,3.0,Tokyo,RJTT,2023-08-04 00:25:00,JL 23,2023-04-08 21:20:05
181,2664,ZYTL,2023-08-04 11:55:00,NaN,Changzhou,ZSCG,2023-08-04 02:10:00,MU 2969,2023-04-08 21:20:05
182,2665,ZYTL,2023-08-04 11:25:00,3.0,Hangzhou,ZSHC,2023-08-04 01:30:00,SC 2111,2023-04-08 21:20:05
